In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))beam

In [2]:
cd /home/dfried/projects/ImageCaptioning.pytorch

/home/dfried/projects/ImageCaptioning.pytorch


In [3]:
import sys

In [4]:
sys.path.append("/home/dfried/projects/ImageCaptioning.pytorch")

In [5]:
import numpy as np
import torch
import einops

In [6]:
import tqdm

In [7]:
import pickle

In [8]:
import argparse
import captioning.utils.opts as opts
import captioning.utils.misc as utils
import captioning.models as models
from captioning.utils import eval_utils

In [9]:
from captioning.data.dataloader import DataLoader
from captioning.data.dataloaderraw import DataLoaderRaw

In [10]:
parser = argparse.ArgumentParser()
opts.add_eval_options(parser)
opts.add_diversity_opts(parser)

In [11]:
opt = parser.parse_args([])

In [12]:
opt

Namespace(batch_size=0, beam_size=1, block_trigrams=0, coco_json='', decoding_constraint=0, diversity_lambda=0.5, dump_images=1, dump_json=1, dump_path=0, eval_oracle=1, group_size=1, id='', image_folder='', image_root='', input_att_dir='', input_box_dir='', input_fc_dir='', input_json='', input_label_h5='', language_eval=0, length_penalty='', max_length=20, num_images=-1, remove_bad_endings=0, sample_method='greedy', sample_n=1, sample_n_method='sample', split='test', suppress_UNK=1, temperature=1.0, verbose_beam=1, verbose_loss=0)

In [13]:
model_fname = 'models/updown/model-best.pth'
infos_fname = 'models/updown/infos_tds-best.pkl'

In [14]:
with open(infos_fname, 'rb') as f:
    infos = utils.pickle_load(f)
infos.keys()

dict_keys(['vocab', 'opt', 'best_val_score', 'iter', 'iterators', 'epoch', 'split_ix'])

In [15]:
replace = ['input_fc_dir', 'input_att_dir', 'input_box_dir', 'input_label_h5', 'input_json', 'batch_size', 'id']
ignore = ['start_from']

for k in vars(infos['opt']).keys():
    if k in replace:
        setattr(opt, k, getattr(opt, k) or getattr(infos['opt'], k, ''))
    elif k not in ignore:
        if not k in vars(opt):
            vars(opt).update({k: vars(infos['opt'])[k]}) # copy over options from model

In [16]:
opt.vocab = infos['vocab']

In [17]:
model = models.setup(opt)

In [18]:
del opt.vocab

In [19]:
CUDA = True

In [20]:
model.load_state_dict(torch.load(model_fname, map_location='cpu'))
if CUDA:
    model.cuda()
model.eval()

UpDownModel(
  (embed): Sequential(
    (0): Embedding(9488, 1000)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (att_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (logit): Linear(in_features=1000, out_features=9488, bias=True)
  (ctx2att): Linear(in_features=1000, out_features=512, bias=True)
  (core): UpDownCore(
    (att_lstm): LSTMCell(3000, 1000)
    (lang_lstm): LSTMCell(2000, 1000)
    (attention): Attention(
      (h2att): Linear(in_features=1000, out_features=512, bias=True)
      (alpha_net): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [21]:
loader = DataLoader(opt, shuffle_override=False, wrap_override=False)

DataLoader loading json file:  data/cocotalk.json
vocab size is  9487
DataLoader loading h5 file:  data/cocobu_fc data/cocobu_att data/cocotalk_box data/cocotalk_label.h5
max sequence length in data is 16
read 123287 image features
assigned 113287 images to split train
assigned 5000 images to split val
assigned 5000 images to split test


In [22]:
vocab = loader.get_vocab()

In [23]:
PAD_ID = 0

In [24]:
REGENERATE = False

In [25]:
split = 'train'

In [26]:
if REGENERATE:
    ixs = []
    ids = []
    file_paths = []
    features = []
    captions = []

    loader.reset_iterator(split)

    for batch_ix in tqdm.trange(len(loader.loaders[split])):
        data = loader.get_batch(split)
        infos = data['infos']
        ixs.extend([d['ix'] for d in infos])
        ids.extend([d['id'] for d in infos])
        file_paths.extend([d['file_path'] for d in infos])
        features.append(data['fc_feats'])

        batch_captions = []
        for batch_labels in data['labels']:
            instance_captions = []
            for img_labels in batch_labels:
                caption = [vocab[str(ix.item())] for ix in img_labels if ix != 0]
                instance_captions.append(caption)
            batch_captions.append(instance_captions)
        captions.extend(batch_captions)
    features_array = torch.cat(features, 0).numpy()

In [27]:
def get_captions_from_batch(data):
    batch_captions = []
    for batch_labels in data['labels']:
        instance_captions = []
        for img_labels in batch_labels:
            caption = [vocab[str(ix.item())] for ix in img_labels if ix != 0]
            instance_captions.append(caption)
        batch_captions.append(instance_captions)
    return batch_captions

In [28]:
fname = 'data/cocobu_fc/all_{}.pkl'.format(split)
if REGENERATE:
    with open(fname, 'wb') as f:
        d = {
            'ixs': ixs,
            'ids': ids,
            'file_paths': file_paths,
            'features': features_array,
            'captions': captions,
        }
        pickle.dump(d, f)
else:
    with open(fname, 'rb') as f:
        d = pickle.load(f)
    ixs, ids, file_paths, features_array, captions = d['ixs'], d['ids'], d['file_paths'], d['features'], d['captions']

In [29]:
import faiss

In [30]:
index = faiss.IndexFlatL2(2048)

In [31]:
index.add(features_array)

In [32]:
index.ntotal

113287

In [33]:
def wrap_tag(tag, inner):
    return f'<{tag}>{inner}</{tag}>'

def image_html(image_path, width=300, border=False):
    if border:
        style = ' style="border: 5px solid #0FF" '
    else:
        style = ''
    return f'<img width={width} src="{image_path}" {style}></img>'

def captions_html(captions):
    #return wrap_tag('p', '<br>'.join(' '.join(cap) for cap in captions))
    return wrap_tag('ol', ''.join(wrap_tag('li', cap) for cap in captions))

def images_html(image_paths, width=300, num_per_row=5, target=None, captions=None):
    rows = []
    for ix in range(0, len(image_paths), num_per_row):
        items = [wrap_tag('td', image_html(image_paths[image_ix], width=width, border=image_ix == target)) 
                 for image_ix in range(ix, ix+num_per_row) if image_ix < len(image_paths)]
        rows.append(wrap_tag('tr', ''.join(items)))
        if captions is not None:
            cap_html = [
                wrap_tag('td', captions_html(captions[image_ix]))
                for image_ix in range(ix, ix+num_per_row)
                if image_ix < len(image_paths)
            ]
            rows.append(wrap_tag('tr', ''.join(cap_html)))
    return wrap_tag('table', ''.join(rows))

def display_images(image_paths, width=300, num_per_row=5, target=None, captions=None):
    display(HTML(images_html(image_paths, width=width, num_per_row=num_per_row, target=target, captions=captions)))

In [34]:
def get_neighbor_batch(img_fc_feat, k, include_self=False, self_ix=None):
    assert len(img_fc_feat.shape) == 1, "should be the features for a single image"
    D, I = index.search(img_fc_feat[None], k)
    n_images, k_ = D.shape
    assert n_images == 1
    indices = []
    if include_self:
        assert self_ix is not None
        indices.append(self_ix)
    indices.extend([ixs[i] for i in I.flatten()])
    data = [loader.dataset[ix, 0, False] for ix in indices]
    batch = loader.dataset.collate_func(data, 'train')
    return batch

In [35]:
def display_neighbors(features, k=5, num_per_row=5):
    neighbor_batch = get_neighbor_batch(features.flatten(), k)
    paths_k = [d['file_path'] for d in neighbor_batch['infos']]
    captions_k = [[' '.join(c) for c in cs] for cs in get_captions_from_batch(neighbor_batch)]
    display_images(paths_k, captions=captions_k, num_per_row=num_per_row)

In [36]:
display_neighbors(features_array[1], k=8, num_per_row=4)

,,,
a young boy standing in front of a computer keyboarda little boy wearing headphones and looking at a computer monitorhe is listening intently to the computer at schoola young boy stares up at the computer monitora young kid with head phones on using a computer,a boy wearing headphones using one computer in a long row of computersa little boy with earphones on listening to somethinga group of people sitting at desk using computerschildren sitting at computer stations on a long tablea small child wearing headphones plays on the computer,some small children playing on laptop games excitedtwo children work at a desk on laptop computerstwo young ladies work on laptops on a white counter topa little girl sitting in front of a laptop computertwo girls who are sitting in front of laptops,a man with glasses sitting at a desktop computertwo men at a computer playing game with headphones ontwo men are wearing headphones and playing a computer gametwo men are in the dark by a laptop computerdark haired man playing a video game on computer
,,,
a man with a hoodie and headphones on in front of a computera person sitting in front of a laptop computer wearing glassesa man with headphones sitting at a desk looking at a computera young man in a red sweatshirt is on the computera man is sitting at the computer desk with a laptop on it,people at a work bench table with laptops and other electronic equipmenttwo people on computers amongst a table full of debristhree people are working on two laptopshands are at work at a table repairing laptop computersa group of people sitting around a pair of laptops,a man holding a smart phone while standing next to a credit card readera man looking at something in his handsa young man is at a workstation with a phonean image of man that is looking at his cellphonea young man is using a cell phone near electronics,two men sitting around a laptop looking at the screena man at a laptop with another looking on at his screentwo men stare intently at a computer screen while one works at the keyboardtwo men at a desk working with a laptop computertwo people looking at a laptop on a desk


In [53]:
# this_pred_captions = []
# for sent in seq:
#     this_pred_captions.append([
#         [model.vocab.get(str(ix.item()), 'IX_{}'.format(ix.item())) for ix in sent if ix.item() != 0]
#     ])

In [54]:
from itertools import groupby

In [55]:
from IPython.core.debugger import set_trace

In [56]:
def caption_scores(fc_feats, att_feats, att_masks, seq, add_bos=True):
    if add_bos:
        seq = torch.cat([torch.zeros(seq.size(0), 1).long().to(seq), seq], 1)
    with torch.no_grad():
        scores = model(fc_feats, att_feats, seq, att_masks)
    mask = (seq[:,:-1] > 0) | (seq[:,1:] > 0)
    # TODO: does this include the EOS score?
    # seq_t: words input at each position, with 0 for pad
    # seq_t: 0, w_0, w_1, w_2, ..., w_k, 0, ...
    # mask : 1, 1  , 1  , 1  , 1  , 1  , 0, ...
    # selected_scores: w_0 + ... + w_k
#     return scores[:,:-1].gather(2, seq[:,1:].unsqueeze(2)).squeeze(2)
    selected_scores = (scores[:,:-1].gather(2, seq[:,1:].unsqueeze(2)).squeeze(2) * mask)
    return selected_scores

In [57]:
def cross_product_scores(fc_feats, att_feats, att_masks, seq, add_bos=True):
    n_captions = seq.size(0)
    n_images = fc_feats.size(0)
    assert att_feats.size(0) == n_images
    assert att_masks.size(0) == n_images
    fc_feats_t = fc_feats.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    att_feats_t = att_feats.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    att_masks_t = att_masks.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    
    fc_feats_t = einops.rearrange(fc_feats_t, 'caps imgs d -> (caps imgs) d')
    att_feats_t = einops.rearrange(att_feats_t, 'caps imgs obj d -> (caps imgs) obj d')
    att_masks_t = einops.rearrange(att_masks_t, 'caps imgs obj -> (caps imgs) obj')
    seq_t = seq.unsqueeze(1).repeat_interleave(n_images, dim=1)
    seq_t = einops.rearrange(seq_t, 'caps imgs d -> (caps imgs) d')
    
    scores_per_timestep = caption_scores(fc_feats_t, att_feats_t, att_masks_t, seq_t, add_bos=add_bos)
    scores = scores_per_timestep.sum(1)
    scores = einops.rearrange(scores, '(caps imgs) -> caps imgs', caps=n_captions, imgs=n_images)
    return scores

In [58]:
pdb on

Automatic pdb calling has been turned ON


In [37]:
loader.reset_iterator('val')

In [61]:
data = loader.get_batch('val')

In [62]:
this_feats = data['fc_feats']
this_ixs = [d['ix'] for d in data['infos']]
this_ids = [d['id'] for d in data['infos']]
this_paths = [d['file_path'] for d in data['infos']]

In [67]:
# set_trace()
sample_n = 10
input_data = data['fc_feats'].cuda(), data['att_feats'].cuda(), data['att_masks'].cuda(), data
n_predictions = []
eval_kwargs = {
    'sample_n_method': 'bs',
    'sample_n': sample_n,
    'temperature': 0.25,
    'verbose': False,
}
eval_utils.eval_split_n(model, n_predictions, input_data=input_data, eval_kwargs=eval_kwargs)
captions_by_id = {}
log_prob_by_id = {}
seq_by_id = {}
for k, ds in groupby(n_predictions, lambda d: d['image_id']):
    ds = list(ds)
    captions_by_id[k] = [d['caption'] for d in ds]
    log_prob_by_id[k] = [d['log_prob'] for d in ds]
    seq_by_id[k] = [d['seq'] for d in ds]
this_captions = [
    captions_by_id[id_]  for id_ in this_ids
]
this_log_probs = [
    log_prob_by_id[id_] for id_ in this_ids
]
this_seq = [
    torch.stack(seq_by_id[id_], 0) for id_ in this_ids
]

In [68]:
K = 5
for i in range(10):
    caps = this_captions[i]
#     this_s0_scores = this_log_probs[i]
    seq = this_seq[i]
    neighbor_batch = get_neighbor_batch(this_feats[i].numpy(), K, include_self=True, self_ix=this_ixs[i])
    
    # num_sampled_captions x (1+K)
    cp_scores = cross_product_scores(
        neighbor_batch['fc_feats'].cuda(),
        neighbor_batch['att_feats'].cuda(), 
        neighbor_batch['att_masks'].cuda(),
        this_seq[i].cuda()
    )
    this_s0_scores = cp_scores[:,0].detach().cpu().tolist()
    
    l1_scores = cp_scores.log_softmax(1)
    s1_scores = l1_scores.log_softmax(0)
    
    this_s1_scores = s1_scores[:,0].detach().cpu().tolist()
    
    def make_strings(lps, caps):
        scored_caps = sorted(list(zip(lps, caps)), reverse=True)
        deduped_caps = [next(g) for k, g in groupby(scored_caps, lambda t: t[1])]
        cap_strings = ["{}  {:.4f}".format(cap, lp) for lp, cap in deduped_caps]
        return cap_strings
    
    display_images([this_paths[i], this_paths[i]], 
                   captions=[
                       make_strings(this_s0_scores, caps)[:10],
                       make_strings(this_s1_scores, caps)[:10],
                   ])
    display_neighbors(data['fc_feats'][i].numpy(), k=min(K, 12), num_per_row=4)
    print()

,
a red and white airplane flying in the sky -5.3846a red and white plane flying in the sky -5.6425a red and white airplane flying in the air -6.2955a red and white airplane flying through a blue sky -6.4906a red and white airplane flying in a blue sky -6.7558a red and white plane flying in a blue sky -6.8799the red and white airplane is flying in the sky -7.9610there is a red and white plane flying in the sky -8.8055an airplane flying in the sky with a red tail -12.0166an airplane is flying in the sky with a red tail -12.4322,a red and white airplane flying in a blue sky -2.2066a red and white airplane flying in the sky -2.2307there is a red and white plane flying in the sky -2.2577the red and white airplane is flying in the sky -2.2766an airplane flying in the sky with a red tail -2.2893an airplane is flying in the sky with a red tail -2.3098a red and white plane flying in a blue sky -2.3132a red and white airplane flying in the air -2.3209a red and white airplane flying through a blue sky -2.4151a red and white plane flying in the sky -2.4287


,
a city street with lots of tall buildings -7.8097a city street with a large building in the background -10.1655a city street with a tall building in the background -10.6663a city street with a large city in the background -10.7126a city street with tall buildings and a large city -11.6744a city street with cars and a tall building -12.1900a city street with tall buildings and a large building -12.2099a city street with tall buildings and a large building in the background -12.6306a city street with cars and people walking on the sidewalk -13.4368a city street with a tall building and a large city -15.2348,a city street with tall buildings and a large building in the background -1.9513a city street with a tall building and a large city -1.9794a city street with tall buildings and a large city -2.0439a city street with tall buildings and a large building -2.1185a city street with a large city in the background -2.2861a city street with a tall building in the background -2.4150a city street with a large building in the background -2.5029a city street with lots of tall buildings -2.5392a city street with cars and a tall building -2.5657a city street with cars and people walking on the sidewalk -3.1856


,
a bus with a advertisement on the side of it -9.9457a truck with a mural on the side of it -10.0262a bus with a mural on the side of it -10.1005a blue truck with a advertisement on the side of it -10.2522a blue bus with a advertisement on the side of it -10.5892a blue truck with a sign on the side of it -11.0073a bus with a large advertisement on the side of it -11.2638a blue truck with a blue and white bus -14.9023a blue truck with a blue and white bus on the side -16.5830a blue truck with a blue and white bus on the side of it -17.4545,a blue truck with a advertisement on the side of it -1.9789a blue truck with a blue and white bus on the side of it -2.0041a blue truck with a blue and white bus on the side -2.0087a blue truck with a blue and white bus -2.1525a truck with a mural on the side of it -2.2120a blue truck with a sign on the side of it -2.2145a blue bus with a advertisement on the side of it -2.3565a bus with a mural on the side of it -2.7477a bus with a large advertisement on the side of it -2.9925a bus with a advertisement on the side of it -3.0165


,
a couple of giraffe standing next to each other -5.7969a couple of giraffe standing on top of a lush green field -6.3289a couple of giraffes are standing in a field -6.6013a couple of giraffe standing next to each other on a field -6.6032two giraffes are standing in a grassy field -6.6232two giraffes standing in the grass near trees -7.4514two giraffes standing in a field of grass -8.3326two giraffes standing in a grassy field next to trees -8.4241two giraffes standing in a field with trees -8.5482two giraffes standing in a grassy field with trees -8.6169,two giraffes standing in a grassy field with trees -1.8652two giraffes standing in the grass near trees -1.9938two giraffes are standing in a grassy field -2.0416two giraffes standing in a grassy field next to trees -2.0757two giraffes standing in a field of grass -2.2234two giraffes standing in a field with trees -2.2874a couple of giraffe standing on top of a lush green field -2.4703a couple of giraffes are standing in a field -2.8429a couple of giraffe standing next to each other -2.9213a couple of giraffe standing next to each other on a field -3.0531


,
a giraffe is laying down in the dirt -5.5802the giraffe is laying down in the dirt -6.8027a giraffe laying on the ground next to a building -7.1025a giraffe laying down in the dirt near a building -7.6570a giraffe laying down in the dirt next to a building -8.2647a giraffe laying on the ground in a zoo -8.3065a giraffe laying on the ground in a dirt -8.6407a giraffe laying down in the dirt in a zoo -8.9004a giraffe laying down on the ground in a zoo -9.0528a giraffe laying down on the ground in a dirt -9.4864,a giraffe laying down in the dirt near a building -2.1925a giraffe laying down in the dirt next to a building -2.1933a giraffe laying on the ground next to a building -2.1935a giraffe laying down on the ground in a dirt -2.3129the giraffe is laying down in the dirt -2.3140a giraffe laying down on the ground in a zoo -2.3565a giraffe laying on the ground in a dirt -2.3627a giraffe laying down in the dirt in a zoo -2.3633a giraffe laying on the ground in a zoo -2.3737a giraffe is laying down in the dirt -2.3938


,
a couple of giraffe standing next to each other -6.3707a couple of giraffe standing on top of a lush green field -6.4391a couple of giraffes are standing in a field -7.0572a couple of giraffe standing next to each other on a lush green field -7.4226two giraffes standing in the grass near trees -8.1003two giraffes standing in a grassy area next to trees -8.6406two giraffes standing in a field next to trees -9.1155two giraffes standing in a field with trees -9.2221two giraffes standing in a grassy field next to trees -9.4280two giraffes standing in a field of grass and trees -9.8691,two giraffes standing in a grassy area next to trees -1.8909two giraffes standing in the grass near trees -1.9922two giraffes standing in a field of grass and trees -2.0097two giraffes standing in a field with trees -2.1959two giraffes standing in a grassy field next to trees -2.1976two giraffes standing in a field next to trees -2.2443a couple of giraffe standing on top of a lush green field -2.4730a couple of giraffe standing next to each other on a lush green field -2.6140a couple of giraffes are standing in a field -2.8668a couple of giraffe standing next to each other -3.3280


,
a fire hydrant with a hose attached to it -7.4004a fire hydrant is on the side of the road -8.1863a fire hydrant that is on the side of the road -8.7289a fire hydrant that is on the side of a road -9.0019the fire hydrant is on the side of the road -9.0924a fire hydrant is painted white and blue -10.1041a fire hydrant is painted white and black -10.1185a fire hydrant is on the sidewalk next to a car -10.4407a fire hydrant is on a sidewalk next to a car -10.7089a fire hydrant that has been painted white and blue -11.2274,a fire hydrant is on the sidewalk next to a car -1.4475a fire hydrant is on a sidewalk next to a car -1.4559a fire hydrant with a hose attached to it -1.5761a fire hydrant is painted white and black -2.6243a fire hydrant that has been painted white and blue -2.8812a fire hydrant is on the side of the road -2.9437the fire hydrant is on the side of the road -3.1696a fire hydrant that is on the side of a road -3.2031a fire hydrant that is on the side of the road -3.2978a fire hydrant is painted white and blue -3.7296


,
a red fire hydrant sitting in the middle of a sidewalk -7.0782a red fire hydrant in front of a tree -7.2148a red fire hydrant in the middle of a park -7.5111a fire hydrant in the middle of a park -7.6126a red fire hydrant in the middle of a sidewalk -7.9263a fire hydrant in the middle of a sidewalk -8.0315a fire hydrant is spraying water onto a street -8.3755a red fire hydrant is in the middle of a sidewalk -9.2053a red fire hydrant in a city street -9.6061a red fire hydrant in a park next to a tree -10.1319,a red fire hydrant in a park next to a tree -2.0400a red fire hydrant in a city street -2.0684a red fire hydrant in front of a tree -2.0851a red fire hydrant is in the middle of a sidewalk -2.0884a red fire hydrant in the middle of a park -2.0914a red fire hydrant in the middle of a sidewalk -2.1153a red fire hydrant sitting in the middle of a sidewalk -2.1219a fire hydrant in the middle of a park -2.7492a fire hydrant in the middle of a sidewalk -3.1231a fire hydrant is spraying water onto a street -3.7515


,
a yellow and blue fire hydrant sitting on the side of a road -4.9590a yellow fire hydrant sitting on the side of a road -5.1579a blue and yellow fire hydrant sitting on the side of a road -5.4978a yellow and blue fire hydrant on a sidewalk -5.8938a blue and yellow fire hydrant on a sidewalk -6.5236the fire hydrant is on the side of the road -8.2083a fire hydrant on a sidewalk next to a street -8.2193a yellow fire hydrant on a sidewalk next to a street -8.2990a fire hydrant on a sidewalk near a street -8.3000a yellow fire hydrant on a sidewalk near a street -8.9389,a blue and yellow fire hydrant sitting on the side of a road -1.7379a blue and yellow fire hydrant on a sidewalk -1.9939a yellow and blue fire hydrant sitting on the side of a road -2.1876a yellow fire hydrant sitting on the side of a road -2.2820a yellow fire hydrant on a sidewalk near a street -2.4456a yellow fire hydrant on a sidewalk next to a street -2.4725a yellow and blue fire hydrant on a sidewalk -2.5455a fire hydrant on a sidewalk next to a street -2.5564the fire hydrant is on the side of the road -2.5935a fire hydrant on a sidewalk near a street -2.6334


,
a close up of a street sign with a sky background -5.2408a street sign that is on a pole -9.8394a street sign with a sticker on it -10.0734a street sign that says UNK and UNK -11.1867a street sign with a sign on it -12.2090a street sign with a sign that reads UNK -13.1063a sign that says UNK and UNK UNK -13.5002a street sign with a sign that says UNK -13.5340a street sign with a sticker of a man on it -14.0173a sign that says UNK UNK and a street -15.6800,a sign that says UNK UNK and a street -1.8236a street sign with a sticker of a man on it -2.0566a sign that says UNK and UNK UNK -2.0826a close up of a street sign with a sky background -2.1726a street sign with a sticker on it -2.3357a street sign with a sign that says UNK -2.3788a street sign with a sign that reads UNK -2.3977a street sign that is on a pole -2.7060a street sign with a sign on it -2.7338a street sign that says UNK and UNK -2.8123
